In [1]:
# Loading dataset
import pandas as pd
dataset = "madeline"

X = pd.read_csv('../data/%s' % dataset + '/%s_train.data' % dataset, header=None, sep=' ')
# For unknown for me reason, X is read with last column filled with NaN
X.drop(X.columns[[-1]], axis=1, inplace=True)
Y = pd.read_csv('../data/%s' % dataset + '/%s_train.solution' % dataset, header=None, sep=' ')[0]

      0    1    2    3    4    5    6    7    8    9   ...   249  250  251  \
0     466  482  468  478  500  475  473  508  513  478 ...   526  522  424   
1     477  495  502  519  480  473  471  496  509  477 ...   495  529  483   
2     453  531  476  487  493  481  475  497  505  477 ...   496  478  529   
3     514  484  437  488  464  471  464  471  486  479 ...   468  463  474   
4     487  491  516  497  486  484  486  462  506  474 ...   487  508  482   
5     486  466  479  493  488  474  515  483  517  477 ...   478  422  418   
6     530  531  516  498  496  482  487  482  475  478 ...   476  523  482   
7     479  484  496  494  513  485  469  472  455  479 ...   479  431  431   
8     525  484  499  477  472  481  493  474  506  480 ...   481  522  457   
9     475  561  551  490  506  480  435  499  514  473 ...   476  426  351   
10    512  475  484  531  441  478  442  495  482  476 ...   476  491  480   
11    447  506  480  464  530  481  486  485  503  479 ...   488

In [10]:
import sklearn
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectPercentile

from libscores import bac_cv
from classifier import eliminate_features

for p in [5, 10, 25]:
    for e in [100, 200, 300]:
        M = Pipeline([
            ('feature_selection', SelectPercentile(percentile=p, score_func=sklearn.feature_selection.f_classif)),
            ('classification', RandomForestClassifier(n_estimators=e, random_state=1, n_jobs=-1))
        ]).fit(X, Y)
        print "SELECT+RF percentile=%d" % p + " n_estimators=%d" % e + ": BAC=%f" % bac_cv(M, X, Y)

for p in [5, 10, 25]:
    for e in [100, 200, 300]:
        M = Pipeline([
            ('feature_selection', SelectPercentile(percentile=p, score_func=sklearn.feature_selection.f_classif)),
            ('classification', ExtraTreesClassifier(n_estimators=e, n_jobs=-1, max_depth=None, min_samples_split=1, random_state=1))
        ]).fit(X, Y)
        print "SELECT+EXTRA_TREE percentile=%d" % p + " n_estimators=%d" % e + ": BAC=%f" % bac_cv(M, X, Y)
        

for f in [20, 50]:
    for e in [100, 200, 300]:
        X_new = X[eliminate_features(X, Y, f)]
        M = Pipeline([
            ('classification', RandomForestClassifier(n_estimators=e, random_state=1, n_jobs=-1))
        ]).fit(X_new, Y)
        print "SELECT_TREE+RF features=%d" % f + " n_estimators=%d" % e + ": BAC=%f" % bac_cv(M, X_new, Y)


for f in [20, 50]:
    for e in [100, 200, 300]:
        X_new = X[eliminate_features(X, Y, f)]
        M = Pipeline([
            ('classification', ExtraTreesClassifier(n_estimators=e, n_jobs=-1, max_depth=None, min_samples_split=1, random_state=1))
        ]).fit(X_new, Y)
        print "SELECT_TREE+EXTRA_TREE features=%d" % f + " n_estimators=%d" % e + ": BAC=%f" % bac_cv(M, X_new, Y)